# A2 — Regressão de Deriva da IMU (Fácil)

Estime a magnitude do bias (deriva) do giroscópio a partir de janelas curtas de IMU. Prefira **feature map quântico + ridge** ou **regressor VQC**.

### Contexto
Uma IMU (Unidade de Medição Inercial) fornece sensoriamento de movimento em 6 eixos: acelerômetro 3 eixos $(a_x,a_y,a_z)$ e giroscópio 3 eixos $(g_x,g_y,g_z)$. Giroscópios medem velocidade angular, mas sensores reais sofrem com um offset lentamente variável (bias) que acumula erro quando integrado.

#### O que é bias/deriva do giroscópio?
Modelo simples: $g_{\text{meas}}(t) = g_{\text{true}}(t) + b_g + \eta(t)$, onde $b_g$ é um bias quase constante e $\eta$ é ruído. Mesmo um bias pequeno (ex.: $0.01\text{–}0.03$ rad/s) causa deriva perceptível de rumo ao longo de minutos.

#### Por que estimar o bias?
- Estabilizar estimativas de atitude/rumo em odometria por rodas/pernas e VIO.
- Melhorar estimação de estado em períodos de baixa excitação (parado/pairando).
- Reduzir frequência de recalibração e permitir compensação online em controladores.

#### Dataset e alvo
Cada amostra é uma janela curta $(W\!\times\!6)$ da IMU. O alvo é a magnitude escalar do bias do giroscópio $\lVert b_g \rVert$ sintetizada por [`common.data_utils.imu_drift_dataset`](common/data_utils.py). O comprimento da janela ($W=32$) mantém custo computacional e contagem de qubits gerenciáveis.

#### Por que métodos quânticos aqui?
Após comprimir janelas em estatísticas robustas, a regressão pode se beneficiar de feature maps não lineares expressivos. Feature maps quânticos ou pequenos VQCs atuam como enriquecedores de kernel sob orçamento reduzido de qubits, potencialmente melhorando generalização em regime de poucos dados.

## Especificação de Entrada e Saída
**Entrada:** `X_train (N×W×6)`, `y_train (N,)`, `X_test (M×W×6)` com $W=32$ nos dados fornecidos  
**Saída:** `y_pred (M,)` magnitude do bias em rad/s  
**Tipo retornado:** array float (ex.: `np.float64`)

Faixa esperada real do bias ≈ $0.01$–$0.03$ rad/s; previsões fora de $[0.0, 0.05]$ são possivelmente errôneas.  
Orientações de recursos: qubits ≤ 10, passos do otimizador ≤ 150; evitar loops Python pesados por amostra.

### Dicas de engenharia de características
- O bias aparece como offset quase constante nos canais do giroscópio (índices 3–5).
- A média temporal dos eixos do giroscópio é forte estimador; variância ajuda a separar ruído vs. deriva.
- Extraia (média, mediana, desvio padrão, MAD) por eixo do giroscópio; agregue estatísticas do acelerômetro para reduzir correlação com movimento.
- Abordagem quântica: condense estatísticas em embedding de baixa dimensão e aplique um feature map quântico antes de regressão linear.

### Notas de pré-processamento
- Padronize (zero média, desvio padrão 1) as features antes da regressão para estabilizar a ridge.
- Evite vazamento de estatísticas do teste: ajuste scalers só em treino.
- Mantenha qubits modestos (≤10) reduzindo para vetor compacto antes da codificação quântica.

## Setup

In [ ]:
import sys
import os
# Adiciona o diretório pai (qhacka-2025) ao caminho do Python
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
import numpy as np
from common import data_utils as du
from common import baselines as bl  
from common import quantum_utils as qu 
np.random.seed(1337)

## Baseline (referência)

_Orientação de baseline:_ em seeds variados com [`common.data_utils.imu_drift_dataset`](common/data_utils.py) o ridge clássico tipicamente obtém MAE $0.012\text{–}0.018$ rad/s; o limite público em $0.020$ deixa margem para melhorias quânticas. Mantenha previsões dentro do intervalo plausível $[0.0,0.05]`.

In [2]:
Xtr, ytr = du.imu_drift_dataset(n=320, w=32, seed=0)
Xte, yte = du.imu_drift_dataset(n=80,  w=32, seed=5)
yp = bl.baseline_imu_reg(Xtr, ytr, Xte)
mae = np.mean(np.abs(yp - yte))
print(f"Baseline MAE: {mae:.4f} rad/s")


Baseline MAE: 0.0049 rad/s


## Sua tarefa: implementar `solve(...)`

In [ ]:
def solve(Xtr, ytr, Xte):
    
    # --- Início da Solução VQC para Regressão ---

    # 1. Importar bibliotecas
    import pennylane as qml
    from pennylane import numpy as pnp
    from sklearn.preprocessing import StandardScaler
    from sklearn.decomposition import PCA
    from common import quantum_utils as qu

    # 2. Engenharia de Features (Clássica)
    # Xtr tem shape (N, 32, 6)
    def extract_features(X):
        # Média e desvio padrão ao longo do eixo do tempo (axis=1)
        mean = X.mean(axis=1)
        std = X.std(axis=1)
        # Concatena em um vetor de 12 features (6 médias + 6 desvios)
        return pnp.concatenate([mean, std], axis=1)

    Xtr_feat = extract_features(Xtr)
    Xte_feat = extract_features(Xte)

    # 3. Pré-processamento (Scaler + PCA)
    # 6 qubits para velocidade
    n_qubits = 6
    
    # Normalizar features X
    x_scaler = StandardScaler()
    Xtr_norm = x_scaler.fit_transform(Xtr_feat)
    Xte_norm = x_scaler.transform(Xte_feat)

    pca = PCA(n_components=n_qubits)
    Xtr_pca = pca.fit_transform(Xtr_norm)
    Xte_pca = pca.transform(Xte_norm)

    # 4. Normalizar Alvos Y (MUITO IMPORTANTE)
    # O VQC retorna valores em [-1, 1]. Nossos alvos 'y' (ytr) estão em [0.01, 0.03].
    # Vamos normalizar 'ytr' para que tenha média 0 e desvio 1.
    y_scaler = StandardScaler()
    # .reshape(-1, 1) é necessário para o scaler
    ytr_norm = y_scaler.fit_transform(ytr.reshape(-1, 1)).flatten()

    # 5. Configurar o VQC (usando o utilitário do hackathon)
    n_layers = 2
    # `make_classifier` nos dá um VQC perfeito para regressão
    # (ele retorna um único valor de -1 a 1)
    circuit, weight_shapes, dev = qu.make_classifier(n_qubits=n_qubits, n_layers=n_layers, shots=None)

    # 6. Definir a Função de Custo (Erro Quadrático Médio)
    # O primeiro argumento (argnum=0) são os 'weights'
    def cost(weights, x_sample, y_target_sample):
        # O circuito retorna um valor em [-1, 1]
        pred = circuit(x_sample, weights) 
        # Comparamos com o y normalizado
        loss = (pred - y_target_sample)**2 
        return loss

    # 7. Treinamento
    n_steps = 150 #
    
    weights = pnp.random.random(size=weight_shapes["weights"], requires_grad=True)
    opt = qml.AdamOptimizer(stepsize=0.01)
    
    # Criamos a função de gradiente para o argumento 0 (weights)
    grad_fn = qml.grad(cost, argnum=0)

    for i in range(n_steps):
        idx = pnp.random.randint(0, len(Xtr_pca))
        x_sample = Xtr_pca[idx]
        y_target_sample = ytr_norm[idx]
        
        # Forçamos o Autograd a não treinar os dados
        x_sample = pnp.array(x_sample, requires_grad=False)
        y_target_sample = pnp.array(y_target_sample, requires_grad=False)
        
        grads = grad_fn(weights, x_sample, y_target_sample)
        weights = opt.apply_grad(grads, weights)

    # 8. Previsão
    preds_norm = []
    for x_sample in Xte_pca:
        pred = circuit(x_sample, weights)
        preds_norm.append(pred)
        
    preds_norm_array = pnp.array(preds_norm).reshape(-1, 1)
    
    # 9. Desfazer a Normalização
    # Convertemos as previsões [-1, 1] de volta para a escala original [0.01, 0.03]
    y_pred = y_scaler.inverse_transform(preds_norm_array)
    
    return y_pred.flatten() # Retorna o array 1D

    # --- Fim da Solução VQC ---

## Testes públicos (rápidos)

In [4]:
Xtr, ytr = du.imu_drift_dataset(n=320, w=32, seed=10)
Xte, yte = du.imu_drift_dataset(n=80,  w=32, seed=11)
yp = solve(Xtr, ytr, Xte)
mae = np.mean(np.abs(yp - yte))
print("Public MAE:", mae)
assert yp.shape == yte.shape
assert mae <= 0.020, "MAE público acima do limiar"
print("OK")


Public MAE: 0.005137499313060666
OK


> Testes adicionais serão executados pelos organizadores com seeds/tamanhos diferentes.

In [5]:
# --- CÉLULA DE VALIDAÇÃO (seed=99) ---
print("--- Iniciando Teste de Validação (Robustedez) ---")

# Usar um seed diferente (ex: 99) para simular os testes ocultos da banca
Xtr_val, ytr_val = du.imu_drift_dataset(n=320, w=32, seed=99)
Xte_val, yte_val = du.imu_drift_dataset(n=80,  w=32, seed=100)

# Rodar a mesma solução 'solve'
yp_val = solve(Xtr_val, ytr_val, Xte_val)
mae_val = np.mean(np.abs(yp_val - yte_val))

print("MAE Validação (seed=99):", mae_val)

# Pegar o MAE do teste público (variável 'mae' da célula anterior)
try:
    print("MAE Público   (seed=10):", mae) 
    print(f"Diferença: {abs(mae - mae_val):.6f}")
    
    # Veredito: 1. A validação tem que passar na meta. 2. A diferença tem que ser pequena.
    assert mae_val <= 0.020, "MAE de validação falhou no limiar!"
    assert abs(mae - mae_val) < 0.01, "Overfitting detectado! A diferença é muito alta."
    print("VEREDITO: Solução robusta, não houve overfitting.")
except NameError:
    print("Execute a célula de Teste Público primeiro para comparar.")

print("--- Fim do Teste de Validação ---")

--- Iniciando Teste de Validação (Robustedez) ---
MAE Validação (seed=99): 0.0044919906000840375
MAE Público   (seed=10): 0.005137499313060666
Diferença: 0.000646
VEREDITO: Solução robusta, não houve overfitting.
--- Fim do Teste de Validação ---
